In [1]:
import pickle
import warnings

import numpy as np
import pandas as pd

from optimizers.qbo import QBO
from solvers.settings import RffWeightedGPExperimentSetting

warnings.filterwarnings('ignore')


In [2]:
def f_3(n):
    dim1 = np.linspace(0, 1, n)
    dim2 = np.linspace(0, 1, n)
    dim3 = np.linspace(0, 1, n)

    domain = []
    for a in dim1:
        for b in dim2:
            for c in dim3:
                domain.append([a, b, c])
    domain = np.array(domain)
    return domain


sizes_3 = {
    "27": f_3(3),
    "512": f_3(8),
    "1000": f_3(10),
    "8000": f_3(20),
    "15625": f_3(25),
    "27000": f_3(30),
    "64000": f_3(40),
    "125000": f_3(50),

}

In [3]:
def f_4(n):
    dim1 = np.linspace(0, 1, n)
    dim2 = np.linspace(0, 1, n)
    dim3 = np.linspace(0, 1, n)
    dim4 = np.linspace(0, 1, n)

    domain = []
    for a in dim1:
        for b in dim2:
            for c in dim3:
                for d in dim4:
                    domain.append([a, b, c, d])

    domain = np.array(domain)
    return domain


sizes_4 = {
    "10000": f_4(10),
}

In [4]:


setting = RffWeightedGPExperimentSetting(
    X_GRID=f_4(10),
    INIT_POINTS=2,
)

optimizer = QBO(num_iter=15, estimate_function="quantum", solver_instance="weighted_rff_pca",
                experiment_settings=setting)


def f(w, x, y, z):
    return np.sin(np.pi * w) + np.sin(np.pi * x) + np.sin(np.pi * y) + np.sin(np.pi * z)  # + x ** 2 + y ** 2 + z ** 2
    # return x ** 2 + y ** 2 + z ** 2


bounds = {'w': (0, 1),
          'x': (0, 1),
          'y': (0, 1),
          'z': (0, 1)}

# optimizer.find_maximum(f, bounds)

In [5]:
# import numpy as np
# 
# from solvers import RffWeightedGPSolver
# 
# 
# class TSRffWeightedGPSolver2(RffWeightedGPSolver):
#     def __init__(self, number_of_dim, samples, settings):
#         super().__init__(number_of_dim, samples, settings)
# 
#     def _cos(self, X):
#         prior_mu, sigma_inv = self._get_prior(np.array(self.X), np.array(self.y))
#         # print("A", prior_mu.shape, sigma_inv.shape)
#         try:
#             prob = np.random.multivariate_normal(prior_mu.flatten(), sigma_inv)
#         except np.linalg.LinAlgError:
#             print(np.array(self.X), np.array(self.y), prior_mu, sigma_inv)
#             prob = np.random.multivariate_normal(prior_mu.flatten(), sigma_inv + 1e-6 * np.eye(sigma_inv.shape[0]))
#         phi = self._get_phi(X, weights=False)
# 
#         mean = np.squeeze(phi @ prob)
#         std = np.einsum('ij,jk,ki->i', phi, sigma_inv, phi.T)
#         std = np.sqrt(self.lambda_ * std)
#         return mean, std
# 
#     def _fit(self):
#         self.mu, self.std = self._cos(self.X_grid)
# 
#     def best_point(self):
#         id_max = np.argmax(self.mu)
#         new_we = self.std[id_max] / np.sqrt(self.lambda_)
#         self.weights = np.append(self.weights, new_we)
#         return self.X_grid[id_max]


In [6]:
def f_2(n):
    dim1 = np.linspace(0, 1, n)
    dim2 = np.linspace(0, 1, n)

    domain = []
    for a in dim1:
        for b in dim2:
            domain.append([a, b])
    domain = np.array(domain)
    return domain


sizes_2 = {
    "25": f_2(5),
    "625": f_2(25),
    "2500": f_2(50),
    "5625": f_2(75),
    "10000": f_2(100),
    "15625": f_2(125),
    "22500": f_2(150),
    "40000": f_2(200),
    "62500": f_2(250),
    "90000": f_2(300),
}

In [7]:
from problems.example_problem import ExampleProblem
from problems.example_problem_xgboost import ExampleProblemXGBoost

problem_size = 2
# example_problem = ExampleProblemXGBoost(problem_size)
example_problem = ExampleProblem()

In [8]:
def get_rff_vectors(settings_rff, number_of_dim):
    W = np.random.multivariate_normal(np.zeros(number_of_dim),
                                      1 / (settings_rff.LS ** 2) * np.identity(number_of_dim), settings_rff.RFF_DIM)
    b = np.random.uniform(0, 2 * np.pi, size=settings_rff.RFF_DIM)
    settings_rff.RFF_W = W
    settings_rff.RFF_B = b
    return settings_rff

In [14]:
problem_domains = {
    2: {
        "625": f_2(25),
        "10000": f_2(100),
        "90000": f_2(300),
    },
    3: {
        "512": f_3(8),
        "15625": f_3(25),
        "125000": f_3(50),
    },
    4: {
        "625": f_4(5),
        "10000": f_4(10),
        "90000": f_4(15),
    }
}

In [15]:
num_iter = 2000
total_runs = 1

In [17]:
for problem_size in [2, 3, 4]:
    for problem_domain in problem_domains[problem_size]:
        for solver in ["weighted_rff", "weighted_rff_ts", "weighted_rff_pca", "weighted_rff_ts_pca", "ucb"]:
            print(problem_size, problem_domain, solver)
            for i in range(total_runs):
                
                problem = ExampleProblemXGBoost(problem_size)
                init_points = problem_size
                
                settings = RffWeightedGPExperimentSetting(BETA=np.sqrt(2))
                if solver == "weighted_rff_pca" or solver == "weighted_rff_ts_pca":
                    settings = get_rff_vectors(settings, 2)
                else:
                    settings = get_rff_vectors(settings, len(example_problem.get_bounds()))
                    
                for estimate_function in ["normal", "quantum"]:
                    file_name = f"results/{solver}_{problem_size}_{estimate_function}_{i}_{pd.Timestamp.now().date()}.pkl"
                    
                    optimizer = QBO(num_iter=num_iter, estimate_function=estimate_function, solver_instance=solver,
                                        experiment_settings=settings, )
                    optimizer.find_maximum(example_problem.reward_function, example_problem.get_bounds())
                    results = {
                            'predicted_values': optimizer.results['history']['predicted_values'],
                            'calls': optimizer.history_quantum,
                            'best': optimizer.results['best'],
                            'time': optimizer.solver.fit_time_history,
                            'settings': settings.model_dump(),
                            'history_parameters': optimizer.results['history']['parameters'],
                            'problem': optimizer.solver.problem,
                        }
                    with open(file_name, 'wb') as handle:
                        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

# for domain_size, domain in list(sizes_4.items()):
#     print(domain_size)
#     for solver in ["weighted_rff_ts_pca"]:
#         for rff_normalized in [False]:
#             for num_qu in [6]:
#                 for i in range(total_runs):
#                     settings = RffWeightedGPExperimentSetting()
# 
#                     if solver == "weighted_rff_pca" or solver == "weighted_rff_ts_pca":
#                         settings = get_rff_vectors(settings, 2)
#                     else:
#                         settings = get_rff_vectors(settings, len(example_problem.get_bounds()))
# 
#                     for estimate_function in ["normal", "quantum"]:
#                         file_name = f"results/{solver}_{domain_size}_{estimate_function}_{i}_{pd.Timestamp.now().date()}.pkl"
# 
#                         if estimate_function == "quantum_noise":
#                             settings.QUANTUM_NOISE = True
#                             estimate_function = "quantum"
#                         else:
#                             settings.QUANTUM_NOISE = False
# 
#                         optimizer = QBO(num_iter=num_iter, estimate_function=estimate_function, solver_instance=solver,
#                                         experiment_settings=settings, )
#                         optimizer.find_maximum(example_problem.reward_function, example_problem.get_bounds())
#                         


2 625 weighted_rff


100%|██████████| 1999/1999 [19:54<00:00,  1.67it/s]
1998it [06:00,  5.55it/s]                          


2 625 weighted_rff_ts


100%|██████████| 1999/1999 [20:13<00:00,  1.65it/s]
1998it [05:44,  5.80it/s]                          


2 625 weighted_rff_pca


...

HERE


1998it [05:30,  6.05it/s]                          


2 625 ucb


 56%|█████▌    | 1120/1999 [47:14<1:53:54,  7.78s/it]

KeyboardInterrupt: 

In [14]:
x = np.array(optimizer.solver.X)
x_t = np.array(optimizer.solver.X_transformed)
y = np.array(optimizer.solver.y)
np.count_nonzero(np.isnan(x)), np.count_nonzero(np.isnan(y)), np.count_nonzero(np.isnan(x_t))

(0, 0, 0)

In [11]:
optimizer.solver.weights[-1]


0.0

In [16]:
phi = optimizer.solver.weights
phi

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
       2.81051749e-08, 3.68222194e-08, 0.00000000e+00])

In [30]:
optimizer.solver.phi_normalized = True

In [31]:
y.shape

(3951,)

In [1]:
from solvers import RffWeightedGPSolver
from solvers.settings import RffWeightedGPExperimentSetting
import numpy as np

settings = RffWeightedGPExperimentSetting()

solver = RffWeightedGPSolver(2, [[np.array([2, 3]), np.array([0.1, 0.2])], [1, 2]], settings)

In [2]:
solver.fit()


In [3]:
solver.best_point()

False 1.1551689963433782 0.6007093323573268 [0.17171717 0.22222222]


array([0.17171717, 0.22222222])